Lambda School Data Science

*Unit 1, Sprint 1, Module 2*

---

# Learning Objectives

- Objective 01 - understand the purpose of feature engineering
- Objective 02 - demonstrate how to work with strings in pandas
- Objective 03 - modify or create dataframe columns using the `apply()` function
- Objective 04 - work with dates and times in pandas


Helpful Links:
- [Minimally Sufficient Pandas](https://medium.com/dunder-data/minimally-sufficient-pandas-a8e67f2a2428)
- [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)
- Python Data Science Handbook
  - [Chapter 3.10](https://jakevdp.github.io/PythonDataScienceHandbook/03.10-working-with-strings.html), Vectorized String Operations
  - [Chapter 3.11](https://jakevdp.github.io/PythonDataScienceHandbook/03.11-working-with-time-series.html), Working with Time Series
- [Lambda Learning Method for DS - By Ryan Herr](https://docs.google.com/document/d/1ubOw9B3Hfip27hF2ZFnW3a3z9xAgrUDRReOEo-FHCVs/edit?usp=sharing)

# [Objective 01](#feature-engineering) - The Purpose of Feature Engineering



## Overview

Feature Engineering is the process of using a combination of domain knowledge, creativity, and the pre-existing columns of a dataset to create completely new columns.

 Machine Learning models try to detect patterns in the data and then associate those patterns with certain predictions. The hope is that by creating new columns on our dataset that we can expose our model to new patterns in the data so that it can make better and better predictions.

This is largely a matter of understanding how to work with individual columns of a dataframe with Pandas --which is what we'll be practicing today!

## Follow Along

Columns of a dataframe hold each hold a specific type of data. Lets inspect some of the common datatypes found in datasets and then we'll make a new feature on a dataset using pre-existing columns.

In [ ]:
import pandas as pd

# Pandas Display Options:


In [ ]:
# Lets take a look at the Ames Iowa Housing Dataset:
df = pd.read_csv('https://raw.githubusercontent.com/ryanleeallred/datasets/master/Ames%20Housing%20Data/train.csv')

### Specific Columns hold specific kinds of data

In [ ]:
print(df.shape)
df.head()

(1460, 81)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [ ]:
df.dtypes

Some columns hold integer values like the `BedroomAbvGr` which stands for "Bedrooms Above Grade." This is the number of non-basement bedrooms in the home.

For more information on specific column meanings view the [data dictionary](https://github.com/ryanleeallred/datasets/blob/master/Ames%20Housing%20Data/data_description.txt).

In [ ]:
# Look at a few rows of the `BedroomAbvGr` column.
# Looks like integers to me!


0       3
1       3
2       3
3       3
4       4
       ..
1455    3
1456    3
1457    4
1458    2
1459    3
Name: BedroomAbvGr, Length: 1460, dtype: int64

What type of variable is BedroomAbvGr?  Is it categorical or quantitative?  If you answered "categorical", is it ordinal, nominal or an identifier?  If you answered "quantitative", is it continuous or discrete?

It is quantitative and discrete.

Some columns hold float values like the `LotFrontage` column.

In [ ]:
# Look at a few rows of the `LotFrontage` column.


0       65.0
1       80.0
2       68.0
3       60.0
4       84.0
        ... 
1455    62.0
1456    85.0
1457    66.0
1458    68.0
1459    75.0
Name: LotFrontage, Length: 1460, dtype: float64

Hmmm, do the values above look like floats to you?

They all have .0 on them so technically they're being stored as floats, but *should* they be stored as floats?

Lets see what all of the possible values for this column are.

Looks to me like the `LotFrontage` column originally held integer values but was cast to a `float` meaning that each original integer values was converted to its corresponding float representation. 

Any guesses as to why that would have happened?


HINT: What's the most common `LotFrontage` value for this column?

In [ ]:
# NaN is the most common value in this column. What is a NaN


`NaN` stands stands for "Not a Number" and is the default missing value indicator with Pandas. This means there were cells in this column that didn't have a LotFrontage value recorded for those homes. 

This is where domain knowledge starts to come in. Think about the context we're working with here: houses. What might a null or blank cell representing "Linear feet of street connected to property" mean in the context of a housing dataset?

Ok, so maybe it makes sense to have some NaNs in this column. What is the datatype of a NaN value?

Perhaps some of this data is truly missing or unrecorded data, but sometimes `NaNs` are more likely to indicate something that was "NA" or "Not Applicable" to a particular observation. There could be multiple reasons why there was no value recorded for a particular feature.

Remember - Pandas tries to maintain a single datatype for all values in a column, and therefore...

In [ ]:
import numpy as np

# What is the datatype of NaN?



float

The datatype of a NaN is float!  This means that if we have a column of integer values, but the column has even a single `NaN` that column will not be treated with the integer datatype but all of the integers will be converted to floats in order to try and preserve the same datatype throughout the entire column.

You can see already how understanding column datatypes is crucial to understanding how Pandas help us manage our data.

### Making new Features

Lets slim down the dataset and consider just a few specific columns:

- `TotalBsmtSF`
- `1stFlrSF`
- `2ndFlrSF`
- `SalePrice1`


In [ ]:
# I can make a smaller dataframe with a few specific column headers
# by passing a list of column headers inside of the square brackets
# get a single column



0        856
1       1262
2        920
3        756
4       1145
        ... 
1455     953
1456    1542
1457    1152
1458    1078
1459    1256
Name: TotalBsmtSF, Length: 1460, dtype: int64

In [ ]:
# Get more than one column, then pass in a list of column headers



,TotalBsmtSF,1stFlrSF
0,856,856
1,1262,1262
2,920,920
3,756,961
4,1145,1145
...,...,...
1455,953,953
1456,1542,2073
1457,1152,1188
1458,1078,1078


In [ ]:
# I can make a smaller dataframe with a few specific column headers
# by passing a list of column headers inside of the square brackets



,TotalBsmtSF,1stFlrSF,2ndFlrSF,SalePrice
0,856,856,854,208500
1,1262,1262,0,181500
2,920,920,866,223500
3,756,961,756,140000
4,1145,1145,1053,250000


In [ ]:
small_df.dtypes

TotalBsmtSF    int64
1stFlrSF       int64
2ndFlrSF       int64
SalePrice      int64
dtype: object

### Syntax for creating new columns

When making a new column on a dataframe, we have to use the square bracket syntax of accessing a column. We can't use "dot syntax" here.

In [ ]:
# Lets add up all of the square footage to get a single square footage 
# column for the entire dataset

# Using bracket syntax to make a new 'TotalSquareFootage' column



,TotalBsmtSF,1stFlrSF,2ndFlrSF,SalePrice,TotalSquareFootage
0,856,856,854,208500,2566
1,1262,1262,0,181500,2524
2,920,920,866,223500,2706
3,756,961,756,140000,2473
4,1145,1145,1053,250000,3343


In [ ]:
# Lets make a nother new column that is 'PricePerSqFt' by
# dividing the price by the square footage



,TotalBsmtSF,1stFlrSF,2ndFlrSF,SalePrice,TotalSquareFootage,price_per_square_ft
0,856,856,854,208500,2566,81.254871
1,1262,1262,0,181500,2524,71.909667
2,920,920,866,223500,2706,82.594235
3,756,961,756,140000,2473,56.611403
4,1145,1145,1053,250000,3343,74.783129


###We can also use if-then statements to create new variables.  

Say we want to categorize houses as having a high price per square foot (greater than or equal to 80 dollars per square foot) or a low price per square foot (less than 80 dollars per square foot).

In [ ]:

#pd.crosstab(small_df['High_ppsqft'],columns='count')

col_0,count
High_ppsqft,
0.0,1128
1.0,332


Now we have added several new columns on our small dataset.

- What does a **high** `PricePerSqFt` say about a home that the square footage and price alone don't capture as directly?

- What does a **low** `PricePerSqFt` say about a home that the square footage and price alone don't directly capture?



### Let's include "and" and "or" conditions in the if-then statements.

Let's identify the 2-story duplex houses.

HouseStyle,1.5Fin,1.5Unf,1Story,2.5Fin,2.5Unf,2Story,SFoyer,SLvl
Two_story_duplex,,,,,,,,
0.0,154,14,726,8,11,436,37,65
1.0,0,0,0,0,0,9,0,0


HouseStyle,1.5Fin,1.5Unf,1Story,2.5Fin,2.5Unf,2Story,SFoyer,SLvl
Two_story_duplex,,,,,,,,
0.0,154,14,726,8,11,436,37,65
1.0,0,0,0,0,0,9,0,0


BldgType,1Fam,2fmCon,Duplex,Twnhs,TwnhsE
Two_story_duplex,,,,,
0.0,1220,31,43,43,114
1.0,0,0,9,0,0


## Challenge

I hope you can see how we have used existing columns to create a new column on a dataset that say something new about our unit of observation. This is what making new features (columns) on a dataset is all about and why it's so essential to data science --particularly predictive modeling "Machine Learning." 

We'll spend the rest of the lecture and assignment today trying to get as good as we can at manipulating (cleaning) and creating new columns on datasets.

# [Objective 02](#work-with-strings) - Work with Strings with Pandas

## Overview

So far we have worked with numeric datatypes (ints and floats) but we haven't worked with any columns containing string values. We can't simply use arithmetic to manipulate string values, so we'll need to learn some more techniques in order to work with this datatype.

## Follow Along

We're going to import a new dataset here to work with. This dataset is from LendingClub and holds information about loans issued in Q4 of 2018. This dataset is a bit messy so it will give us plenty of opportunities to clean up existing columns as well as create new ones.

The `!wget` shell command being used here does exactly the same thing that your browser does when you type a URL in the address. It makes a request or "gets" the file at that address. However, in our case the file isn't a webpage, it's a compressed CSV file. 

Try copying and pasting the URL from below into your browser, did it start an automatic download? Any URLs like this that start automatic downloads when navigated to can be used along with the `!wget` command to bring files directly into your notebook's memory.

### Load a new dataset

In [ ]:
!wget https://resources.lendingclub.com/LoanStats_2018Q4.csv.zip

--2020-09-20 12:06:56--  https://resources.lendingclub.com/LoanStats_2018Q4.csv.zip
Resolving resources.lendingclub.com (resources.lendingclub.com)... 100.20.37.44, 54.244.115.45, 54.148.13.215
Connecting to resources.lendingclub.com (resources.lendingclub.com)|100.20.37.44|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘LoanStats_2018Q4.csv.zip’

LoanStats_2018Q4.cs     [              <=>   ]  22.28M  5.80MB/s    in 4.0s    

2020-09-20 12:07:00 (5.56 MB/s) - ‘LoanStats_2018Q4.csv.zip’ saved [23360898]



We need to use the `!unzip` command to extract the csv from the zipped folder.

In [ ]:
!unzip LoanStats_2018Q4.csv.zip

Archive:  LoanStats_2018Q4.csv.zip
  inflating: LoanStats_2018Q4.csv    


We can also use bash/shell commands to look at the raw file using the `!head` and `!tail` commands

In [ ]:
!head LoanStats_2018Q4.csv

Notes offered by Prospectus (https://www.lendingclub.com/info/prospectus.action)
"id","member_id","loan_amnt","funded_amnt","funded_amnt_inv","term","int_rate","installment","grade","sub_grade","emp_title","emp_length","home_ownership","annual_inc","verification_status","issue_d","loan_status","pymnt_plan","url","desc","purpose","title","zip_code","addr_state","dti","delinq_2yrs","earliest_cr_line","inq_last_6mths","mths_since_last_delinq","mths_since_last_record","open_acc","pub_rec","revol_bal","revol_util","total_acc","initial_list_status","out_prncp","out_prncp_inv","total_pymnt","total_pymnt_inv","total_rec_prncp","total_rec_int","total_rec_late_fee","recoveries","collection_recovery_fee","last_pymnt_d","last_pymnt_amnt","next_pymnt_d","last_credit_pull_d","collections_12_mths_ex_med","mths_since_last_major_derog","policy_code","application_type","annual_inc_joint","dti_joint","verification_status_joint","acc_now_delinq","tot_coll_amt","tot_cur_bal","open_acc_6m","open_act_il","op

In [ ]:
!tail LoanStats_2018Q4.csv

"","","5600","5600","5600"," 36 months"," 13.56%","190.21","C","C1","","n/a","RENT","15600","Not Verified","Oct-2018","Current","n","","","credit_card","Credit card refinancing","836xx","ID","15.31","0","Aug-2012","0","","97","9","1","5996","34.5%","11","w","2449.86","2449.86","4174.07","4174.07","3150.14","1023.93","0.0","0.0","0.0","Aug-2020","190.21","Sep-2020","Aug-2020","0","","1","Individual","","","","0","0","5996","0","0","0","1","20","0","","0","2","3017","35","17400","1","0","0","3","750","4689","45.5","0","0","20","73","13","13","0","13","","20","","0","3","5","4","4","1","9","10","5","9","0","0","0","0","100","25","1","0","17400","5996","8600","0","","","","","","","","","","","","N","","","","","","","","","","","","","","","N","","","","","",""
"","","23000","23000","23000"," 36 months"," 15.02%","797.53","C","C3","Tax Consultant","10+ years","MORTGAGE","75000","Source Verified","Oct-2018","Charged Off","n","","","debt_consolidation","Debt consolidation","352xx","AL","20.

As we look at the raw file itself, do you see anything that might cause us trouble as we read in the CSV file to a dataframe?

In [ ]:
# Read in the CSV

df = pd.read_csv('LoanStats_2018Q4.csv', header=1)

print(df.shape)
df.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(128414, 144)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,...,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,27975.0,27975.0,27975.0,36 months,14.47%,962.52,C,C2,Conductor,10+ years,MORTGAGE,180000.0,Not Verified,Dec-2018,Fully Paid,n,NaN,NaN,credit_card,Credit card refinancing,117xx,NY,11.47,0.0,Jul-1995,0.0,39.0,NaN,10.0,0.0,29711.0,66.8%,19.0,w,0.00,0.00,31804.529849,31804.53,27975.00,3829.53,0.0,0.00,0.0000,Jan-2020,20288.02,NaN,Mar-2020,0.0,...,1.0,8.0,13.0,7.0,10.0,0.0,0.0,0.0,0.0,78.9,60.0,0.0,0.0,286525.0,45387.0,29500.0,25025.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,10000.0,10000.0,10000.0,60 months,12.98%,227.43,B,B5,Printer,9 years,RENT,60000.0,Not Verified,Dec-2018,Current,n,NaN,NaN,credit_card,Credit card refinancing,660xx,KS,14.90,0.0,May-2007,0.0,NaN,112.0,7.0,1.0,10677.0,54.2%,12.0,w,7352.95,7352.95,4487.310000,4487.31,2647.05,1840.26,0.0,0.00,0.0000,Aug-2020,227.43,Sep-2020,Aug-2020,0.0,...,6.0,5.0,6.0,5.0,7.0,0.0,0.0,0.0,1.0,100.0,33.3,1.0,0.0,36200.0,27595.0,13000.0,16500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,28000.0,28000.0,28000.0,60 months,13.56%,645.15,C,C1,Project Manager,10+ years,MORTGAGE,128500.0,Source Verified,Dec-2018,Current,n,NaN,NaN,credit_card,Credit card refinancing,760xx,TX,27.35,1.0,Jul-1987,0.0,24.0,NaN,16.0,0.0,55206.0,75%,31.0,w,20667.91,20667.91,12871.360000,12871.36,7332.09,5539.27,0.0,0.00,0.0000,Aug-2020,645.15,Sep-2020,Aug-2020,0.0,...,9.0,11.0,17.0,11.0,15.0,NaN,0.0,0.0,1.0,94.0,100.0,0.0,0.0,542027.0,128345.0,55000.0,60331.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,16000.0,16000.0,16000.0,60 months,13.56%,368.66,C,C1,LMSW,10+ years,RENT,46000.0,Not Verified,Dec-2018,Current,n,NaN,NaN,credit_card,Credit card refinancing,787xx,TX,11.09,1.0,Oct-1997,1.0,11.0,NaN,9.0,0.0,18946.0,32.1%,21.0,w,11503.84,11503.84,7570.880000,7570.88,4496.16,3074.72,0.0,0.00,0.0000,Aug-2020,368.66,Sep-2020,Aug-2020,0.0,...,10.0,7.0,11.0,3.0,9.0,0.0,0.0,0.0,1.0,94.7,20.0,0.0,0.0,138125.0,128218.0,57600.0,79125.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,7500.0,7500.0,7500.0,36 months,10.72%,244.55,B,B2,Program Manager,2 years,RENT,84000.0,Not Ve

The extra rows at the top and bottom of the file have done two things:

1) The top row has made it so that the entire dataset is being interpreted as column headers

2) The bottom two rows have been read into the 'id' column and are causing every column to have at least two `NaN` values in it.

In [ ]:
# We can fix the header problem by using the 'skiprows' parameter

df = pd.read_csv('LoanStats_2018Q4.csv', skiprows=1)



/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(128414, 144)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,...,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,27975.0,27975.0,27975.0,36 months,14.47%,962.52,C,C2,Conductor,10+ years,MORTGAGE,180000.0,Not Verified,Dec-2018,Fully Paid,n,NaN,NaN,credit_card,Credit card refinancing,117xx,NY,11.47,0.0,Jul-1995,0.0,39.0,NaN,10.0,0.0,29711.0,66.8%,19.0,w,0.00,0.00,31804.529849,31804.53,27975.00,3829.53,0.0,0.00,0.0000,Jan-2020,20288.02,NaN,Mar-2020,0.0,...,1.0,8.0,13.0,7.0,10.0,0.0,0.0,0.0,0.0,78.9,60.0,0.0,0.0,286525.0,45387.0,29500.0,25025.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,10000.0,10000.0,10000.0,60 months,12.98%,227.43,B,B5,Printer,9 years,RENT,60000.0,Not Verified,Dec-2018,Current,n,NaN,NaN,credit_card,Credit card refinancing,660xx,KS,14.90,0.0,May-2007,0.0,NaN,112.0,7.0,1.0,10677.0,54.2%,12.0,w,7352.95,7352.95,4487.310000,4487.31,2647.05,1840.26,0.0,0.00,0.0000,Aug-2020,227.43,Sep-2020,Aug-2020,0.0,...,6.0,5.0,6.0,5.0,7.0,0.0,0.0,0.0,1.0,100.0,33.3,1.0,0.0,36200.0,27595.0,13000.0,16500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,28000.0,28000.0,28000.0,60 months,13.56%,645.15,C,C1,Project Manager,10+ years,MORTGAGE,128500.0,Source Verified,Dec-2018,Current,n,NaN,NaN,credit_card,Credit card refinancing,760xx,TX,27.35,1.0,Jul-1987,0.0,24.0,NaN,16.0,0.0,55206.0,75%,31.0,w,20667.91,20667.91,12871.360000,12871.36,7332.09,5539.27,0.0,0.00,0.0000,Aug-2020,645.15,Sep-2020,Aug-2020,0.0,...,9.0,11.0,17.0,11.0,15.0,NaN,0.0,0.0,1.0,94.0,100.0,0.0,0.0,542027.0,128345.0,55000.0,60331.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,16000.0,16000.0,16000.0,60 months,13.56%,368.66,C,C1,LMSW,10+ years,RENT,46000.0,Not Verified,Dec-2018,Current,n,NaN,NaN,credit_card,Credit card refinancing,787xx,TX,11.09,1.0,Oct-1997,1.0,11.0,NaN,9.0,0.0,18946.0,32.1%,21.0,w,11503.84,11503.84,7570.880000,7570.88,4496.16,3074.72,0.0,0.00,0.0000,Aug-2020,368.66,Sep-2020,Aug-2020,0.0,...,10.0,7.0,11.0,3.0,9.0,0.0,0.0,0.0,1.0,94.7,20.0,0.0,0.0,138125.0,128218.0,57600.0,79125.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,7500.0,7500.0,7500.0,36 months,10.72%,244.55,B,B2,Program Manager,2 years,RENT,84000.0,Not Ve

Lets look at the NaN values of each column so that you can see the problem that the extra rows at the bottom of the file are creating for us

In [ ]:
# Sum null values by column and sort from least to greatest


inq_fi                                             2
mo_sin_old_rev_tl_op                               2
delinq_amnt                                        2
chargeoff_within_12_mths                           2
acc_open_past_24mths                               2
inq_last_12m                                       2
total_cu_tl                                        2
total_rev_hi_lim                                   2
open_rv_24m                                        2
open_rv_12m                                        2
total_bal_il                                       2
open_il_24m                                        2
open_il_12m                                        2
open_act_il                                        2
open_acc_6m                                        2
tot_cur_bal                                        2
tot_coll_amt                                       2
acc_now_delinq                                     2
application_type                              

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,...,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
128409,NaN,NaN,5000.0,5000.0,5000.0,36 months,13.56%,169.83,C,C1,Payoff Clerk,10+ years,MORTGAGE,35360.0,Not Verified,Oct-2018,Current,n,NaN,NaN,debt_consolidation,Debt consolidation,381xx,TN,11.30,1.0,Jun-2006,0.0,21.0,NaN,9.0,0.0,2597.0,27.3%,15.0,f,2187.38,2187.38,3732.49,3732.49,2812.62,919.87,0.0,0.0,0.0,Aug-2020,169.83,Sep-2020,Aug-2020,0.0,...,6.0,6.0,7.0,3.0,9.0,0.0,0.0,0.0,3.0,92.9,50.0,0.0,0.0,93908.0,4976.0,3000.0,6028.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
128410,NaN,NaN,10000.0,10000.0,9750.0,36 months,11.06%,327.68,B,B3,NaN,NaN,RENT,44400.0,Source Verified,Oct-2018,Current,n,NaN,NaN,credit_card,Credit card refinancing,980xx,WA,11.78,0.0,Oct-2008,2.0,40.0,NaN,15.0,0.0,6269.0,13.1%,25.0,f,4285.08,4177.96,7193.60,7013.76,5714.92,1478.68,0.0,0.0,0.0,Aug-2020,327.68,Sep-2020,Aug-2020,0.0,...,3.0,14.0,22.0,4.0,15.0,0.0,0.0,0.0,3.0,92.0,0.0,0.0,0.0,57871.0,16440.0,20500.0,10171.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
128411,NaN,NaN,10000.0,10000.0,10000.0,36 months,16.91%,356.08,C,C5,Key Accounts Manager,2 years,RENT,80000.0,Not Verified,Oct-2018,Current,n,NaN,NaN,other,Other,021xx,MA,17.72,1.0,Sep-2006,0.0,14.0,NaN,17.0,0.0,1942.0,30.8%,31.0,w,4495.59,4495.59,7824.37,7824.37,5504.41,2319.96,0.0,0.0,0.0,Aug-2020,356.08,Sep-2020,Aug-2020,0.0,...,22.0,2.0,9.0,1.0,17.0,0.0,0.0,0.0,1.0,74.2,0.0,0.0,0.0,73669.0,59194.0,4000.0,67369.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
128412,Total amount funded in policy code 1: 2050909275,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
128413,Total amount funded in policy code 2: 820109297,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Na

In [ ]:
# Address the extra NaNs in each column by skipping the footer as well.
df = pd.read_csv('LoanStats_2018Q4.csv', skiprows=1, skipfooter=2, engine='python')



(128412, 144)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,...,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,27975,27975,27975.0,36 months,14.47%,962.52,C,C2,Conductor,10+ years,MORTGAGE,180000.0,Not Verified,Dec-2018,Fully Paid,n,NaN,NaN,credit_card,Credit card refinancing,117xx,NY,11.47,0,Jul-1995,0,39.0,NaN,10,0,29711,66.8%,19,w,0.00,0.00,31804.529849,31804.53,27975.00,3829.53,0.0,0.00,0.0000,Jan-2020,20288.02,NaN,Mar-2020,0,...,1,8,13,7,10,0.0,0,0,0,78.9,60.0,0,0,286525,45387,29500,25025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,10000,10000,10000.0,60 months,12.98%,227.43,B,B5,Printer,9 years,RENT,60000.0,Not Verified,Dec-2018,Current,n,NaN,NaN,credit_card,Credit card refinancing,660xx,KS,14.90,0,May-2007,0,NaN,112.0,7,1,10677,54.2%,12,w,7352.95,7352.95,4487.310000,4487.31,2647.05,1840.26,0.0,0.00,0.0000,Aug-2020,227.43,Sep-2020,Aug-2020,0,...,6,5,6,5,7,0.0,0,0,1,100.0,33.3,1,0,36200,27595,13000,16500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,28000,28000,28000.0,60 months,13.56%,645.15,C,C1,Project Manager,10+ years,MORTGAGE,128500.0,Source Verified,Dec-2018,Current,n,NaN,NaN,credit_card,Credit card refinancing,760xx,TX,27.35,1,Jul-1987,0,24.0,NaN,16,0,55206,75%,31,w,20667.91,20667.91,12871.360000,12871.36,7332.09,5539.27,0.0,0.00,0.0000,Aug-2020,645.15,Sep-2020,Aug-2020,0,...,9,11,17,11,15,NaN,0,0,1,94.0,100.0,0,0,542027,128345,55000,60331,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,16000,16000,16000.0,60 months,13.56%,368.66,C,C1,LMSW,10+ years,RENT,46000.0,Not Verified,Dec-2018,Current,n,NaN,NaN,credit_card,Credit card refinancing,787xx,TX,11.09,1,Oct-1997,1,11.0,NaN,9,0,18946,32.1%,21,w,11503.84,11503.84,7570.880000,7570.88,4496.16,3074.72,0.0,0.00,0.0000,Aug-2020,368.66,Sep-2020,Aug-2020,0,...,10,7,11,3,9,0.0,0,0,1,94.7,20.0,0,0,138125,128218,57600,79125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,7500,7500,7500.0,36 months,10.72%,244.55,B,B2,Program Manager,2 years,RENT,84000.0,Not Verified,Dec-2018,Charged Off,n,NaN,NaN,debt_consolidation,Debt consolidation,600xx,IL,3.86,0,May-2005,1,NaN,114.0,9,1,2200,12.6%,17,w,0.00,0.00,2829.930000,2829.93,1465.61,484.09,0.0,880.23

For good measure, we'll also drop some columns that are made up completely of NaN values.

Why might LendingClub have included columns in their dataset that are 100% blank?

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,...,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,27975,27975,27975.0,36 months,14.47%,962.52,C,C2,Conductor,10+ years,MORTGAGE,180000.0,Not Verified,Dec-2018,Fully Paid,n,NaN,NaN,credit_card,Credit card refinancing,117xx,NY,11.47,0,Jul-1995,0,39.0,NaN,10,0,29711,66.8%,19,w,0.00,0.00,31804.529849,31804.53,27975.00,3829.53,0.0,0.00,0.0000,Jan-2020,20288.02,NaN,Mar-2020,0,...,1,8,13,7,10,0.0,0,0,0,78.9,60.0,0,0,286525,45387,29500,25025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,10000,10000,10000.0,60 months,12.98%,227.43,B,B5,Printer,9 years,RENT,60000.0,Not Verified,Dec-2018,Current,n,NaN,NaN,credit_card,Credit card refinancing,660xx,KS,14.90,0,May-2007,0,NaN,112.0,7,1,10677,54.2%,12,w,7352.95,7352.95,4487.310000,4487.31,2647.05,1840.26,0.0,0.00,0.0000,Aug-2020,227.43,Sep-2020,Aug-2020,0,...,6,5,6,5,7,0.0,0,0,1,100.0,33.3,1,0,36200,27595,13000,16500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,28000,28000,28000.0,60 months,13.56%,645.15,C,C1,Project Manager,10+ years,MORTGAGE,128500.0,Source Verified,Dec-2018,Current,n,NaN,NaN,credit_card,Credit card refinancing,760xx,TX,27.35,1,Jul-1987,0,24.0,NaN,16,0,55206,75%,31,w,20667.91,20667.91,12871.360000,12871.36,7332.09,5539.27,0.0,0.00,0.0000,Aug-2020,645.15,Sep-2020,Aug-2020,0,...,9,11,17,11,15,NaN,0,0,1,94.0,100.0,0,0,542027,128345,55000,60331,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,16000,16000,16000.0,60 months,13.56%,368.66,C,C1,LMSW,10+ years,RENT,46000.0,Not Verified,Dec-2018,Current,n,NaN,NaN,credit_card,Credit card refinancing,787xx,TX,11.09,1,Oct-1997,1,11.0,NaN,9,0,18946,32.1%,21,w,11503.84,11503.84,7570.880000,7570.88,4496.16,3074.72,0.0,0.00,0.0000,Aug-2020,368.66,Sep-2020,Aug-2020,0,...,10,7,11,3,9,0.0,0,0,1,94.7,20.0,0,0,138125,128218,57600,79125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,7500,7500,7500.0,36 months,10.72%,244.55,B,B2,Program Manager,2 years,RENT,84000.0,Not Verified,Dec-2018,Charged Off,n,NaN,NaN,debt_consolidation,Debt consolidation,600xx,IL,3.86,0,May-2005,1,NaN,114.0,9,1,2200,12.6%,17,w,0.00,0.00,2829.930000,2829.93,1465.61,484.09,0.0,880.23

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,...,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,27975,27975,27975.0,36 months,14.47%,962.52,C,C2,Conductor,10+ years,MORTGAGE,180000.0,Not Verified,Dec-2018,Fully Paid,n,credit_card,Credit card refinancing,117xx,NY,11.47,0,Jul-1995,0,39.0,NaN,10,0,29711,66.8%,19,w,0.00,0.00,31804.529849,31804.53,27975.00,3829.53,0.0,0.00,0.0000,Jan-2020,20288.02,NaN,Mar-2020,0,NaN,1,Individual,NaN,...,1,8,13,7,10,0.0,0,0,0,78.9,60.0,0,0,286525,45387,29500,25025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
1,10000,10000,10000.0,60 months,12.98%,227.43,B,B5,Printer,9 years,RENT,60000.0,Not Verified,Dec-2018,Current,n,credit_card,Credit card refinancing,660xx,KS,14.90,0,May-2007,0,NaN,112.0,7,1,10677,54.2%,12,w,7352.95,7352.95,4487.310000,4487.31,2647.05,1840.26,0.0,0.00,0.0000,Aug-2020,227.43,Sep-2020,Aug-2020,0,NaN,1,Individual,NaN,...,6,5,6,5,7,0.0,0,0,1,100.0,33.3,1,0,36200,27595,13000,16500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
2,28000,28000,28000.0,60 months,13.56%,645.15,C,C1,Project Manager,10+ years,MORTGAGE,128500.0,Source Verified,Dec-2018,Current,n,credit_card,Credit card refinancing,760xx,TX,27.35,1,Jul-1987,0,24.0,NaN,16,0,55206,75%,31,w,20667.91,20667.91,12871.360000,12871.36,7332.09,5539.27,0.0,0.00,0.0000,Aug-2020,645.15,Sep-2020,Aug-2020,0,71.0,1,Individual,NaN,...,9,11,17,11,15,NaN,0,0,1,94.0,100.0,0,0,542027,128345,55000,60331,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
3,16000,16000,16000.0,60 months,13.56%,368.66,C,C1,LMSW,10+ years,RENT,46000.0,Not Verified,Dec-2018,Current,n,credit_card,Credit card refinancing,787xx,TX,11.09,1,Oct-1997,1,11.0,NaN,9,0,18946,32.1%,21,w,11503.84,11503.84,7570.880000,7570.88,4496.16,3074.72,0.0,0.00,0.0000,Aug-2020,368.66,Sep-2020,Aug-2020,0,NaN,1,Individual,NaN,...,10,7,11,3,9,0.0,0,0,1,94.7,20.0,0,0,138125,128218,57600,79125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
4,7500,7500,7500.0,36 months,10.72%,244.55,B,B2,Program Manager,2 years,RENT,84000.0,Not Verified,Dec-2018,Charged Off,n,debt_consolidation,Debt consolidation,600xx,IL,3.86,0,May-2005,1,NaN,114.0,9,1,2200,12.6%,17,

### Clean up the `int_rate` column

When we're preparing a dataset for a machine learning model we typically don't want to leave any string values in our dataset --because it's hard to do math on words. 

Specifically, we have a column that is representing a numeric value, but currently doesn't have a numeric datatype. Lets look at the first 10 values of the `int_rate` column:

In [ ]:
# Look at the first 10 values of the int_rate column


0     14.47%
1     12.98%
2     13.56%
3     13.56%
4     10.72%
5     20.89%
6     26.31%
7     23.40%
8     19.92%
9     17.97%
Name: int_rate, dtype: object

In [ ]:
# Look at a specific value from the int_rate column



' 14.47%'

' 13.56%'

Problems that we need to address with this column:

- String column that should be numeric
- Percent sign `%` included with the number
- Leading space at the beginning of the string

However, we're not going to try and write exactly the right code to fix this column in one go. We're going to methodically build up to the code that will help us address these problems.


In [ ]:
# Lets start with just fixing a single string.
# If we can fix one, we can usually fix all of them




In [ ]:
#Remove extra white space




In [ ]:
#Doesn't work if there isn't any extra white space to remove



In [ ]:
#You can "chain" two strip functions to remove both the spaces and the % sign



'14.47'

In [ ]:
# "Cast" the string value to a float
# "cast" -> Change something's data type
# This is the line of code that we're after! ->



14.47

float

### Write a function to make our solution reusable!

0     14.47%
1     12.98%
2     13.56%
3     13.56%
4     10.72%
Name: int_rate, dtype: object

In [ ]:
# Write a function that can do what we have written above to any 
# string that is passsed to it.



In [ ]:
# Test out our function by calling it on our example



14.47

In [ ]:
# is the data type correct?



float

### Apply our solution to every cell in a column

0         14.47
1         12.98
2         13.56
3         13.56
4         10.72
          ...  
128407    15.02
128408    15.02
128409    13.56
128410    11.06
128411    16.91
Name: int_rate, Length: 128412, dtype: float64

In [ ]:
# pass in *only* the name of the function, don't call it. 
# This works because we know the function works on every item in the column
# so I can simply "apply" it to the entire column



,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,...,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,int_rate_cleaned
0,NaN,NaN,27975,27975,27975.0,36 months,14.47%,962.52,C,C2,Conductor,10+ years,MORTGAGE,180000.0,Not Verified,Dec-2018,Fully Paid,n,NaN,NaN,credit_card,Credit card refinancing,117xx,NY,11.47,0,Jul-1995,0,39.0,NaN,10,0,29711,66.8%,19,w,0.00,0.00,31804.529849,31804.53,27975.00,3829.53,0.0,0.00,0.0000,Jan-2020,20288.02,NaN,Mar-2020,0,...,8,13,7,10,0.0,0,0,0,78.9,60.0,0,0,286525,45387,29500,25025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,14.47
1,NaN,NaN,10000,10000,10000.0,60 months,12.98%,227.43,B,B5,Printer,9 years,RENT,60000.0,Not Verified,Dec-2018,Current,n,NaN,NaN,credit_card,Credit card refinancing,660xx,KS,14.90,0,May-2007,0,NaN,112.0,7,1,10677,54.2%,12,w,7352.95,7352.95,4487.310000,4487.31,2647.05,1840.26,0.0,0.00,0.0000,Aug-2020,227.43,Sep-2020,Aug-2020,0,...,5,6,5,7,0.0,0,0,1,100.0,33.3,1,0,36200,27595,13000,16500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,12.98
2,NaN,NaN,28000,28000,28000.0,60 months,13.56%,645.15,C,C1,Project Manager,10+ years,MORTGAGE,128500.0,Source Verified,Dec-2018,Current,n,NaN,NaN,credit_card,Credit card refinancing,760xx,TX,27.35,1,Jul-1987,0,24.0,NaN,16,0,55206,75%,31,w,20667.91,20667.91,12871.360000,12871.36,7332.09,5539.27,0.0,0.00,0.0000,Aug-2020,645.15,Sep-2020,Aug-2020,0,...,11,17,11,15,NaN,0,0,1,94.0,100.0,0,0,542027,128345,55000,60331,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,13.56
3,NaN,NaN,16000,16000,16000.0,60 months,13.56%,368.66,C,C1,LMSW,10+ years,RENT,46000.0,Not Verified,Dec-2018,Current,n,NaN,NaN,credit_card,Credit card refinancing,787xx,TX,11.09,1,Oct-1997,1,11.0,NaN,9,0,18946,32.1%,21,w,11503.84,11503.84,7570.880000,7570.88,4496.16,3074.72,0.0,0.00,0.0000,Aug-2020,368.66,Sep-2020,Aug-2020,0,...,7,11,3,9,0.0,0,0,1,94.7,20.0,0,0,138125,128218,57600,79125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,13.56
4,NaN,NaN,7500,7500,7500.0,36 months,10.72%,244.55,B,B2,Program Manager,2 years,RENT,84000.0,Not Verified,Dec-2018,Charged Off,n,NaN,NaN,debt_consolidation,Debt consolidation,600xx,IL,3.86,0,May-2005,1,NaN,114.0,9,1,2200,12.6%,17,w,0.00,0.00,2829.930000,2829.93,146

In [ ]:

# What type of data is held in our new column?

# Look at the datatypes of the last 5 columns



settlement_date           object
settlement_amount        float64
settlement_percentage    float64
settlement_term          float64
int_rate_cleaned         float64
dtype: object

## Challenge

We can create a new column with our cleaned values or overwrite the original, whatever we think best suits our needs. On your assignment you will take the same approach in trying to methodically build up the complexity of your code until you have a few lines that will work for any cell in a column. At that point you'll contain all of that functionality in a reusable function block and then use the `.apply()` function to... well... apply those changes to an entire column.

# [Objective 03](#pandas-apply) - Modify and Create Columns using `.apply()`



## Overview

We're already seen one example of using the `.apply()` function to clean up a column. Lets see if we can do it again, but this time on a slightly more complicated use case.

Remember, the goal here is to write a function that will work correctly on any **individual** cell of a specific column. Then we can reuse that function on those individual cells of a dataframe column via the `.apply()` function.

Lets clean up the emp_title "Employment Title" column!

## Follow Along

First we'll try and diagnose how bad the problem is and what improvements we might be able to make.

In [ ]:
# Look at the top 20 employment titles

df['emp_title'].value_counts(dropna=False).head(20)

NaN                   20947
Teacher                2090
Manager                1773
Registered Nurse        952
Driver                  924
RN                      726
Supervisor              697
Sales                   580
Project Manager         526
General Manager         523
Office Manager          521
Owner                   420
Director                402
Operations Manager      387
Truck Driver            387
Nurse                   326
Engineer                325
Sales Manager           304
manager                 301
Supervisor              270
Name: emp_title, dtype: int64

In [ ]:
# How many different unique employment titles are there currently?



43892

In [ ]:
# How often is the employment_title null?


20947

What are some possible reasons as to why a person's employment title may have not been provided?

In [ ]:
# Create some examples that represent the cases that we want to clean up
import numpy as np


In [ ]:
# Write a function to clean up these use cases and increase uniformity.



In [ ]:
  # print things that you want to force to be shown in the notebook output
 

In [ ]:
# The value of the last thing to be returned gets printed out


'Teacher'

In [ ]:
# if the last thing to be returned gets saved to a variable
# then it won't get printed out


In [ ]:
# if I include just a variable name, it will get printed out



'Teacher'

In [ ]:
#Apply the function to every row of the emp_title variable



,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,...,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,emp_title_cleaned
0,27975,27975,27975.0,36 months,14.47%,962.52,C,C2,Conductor,10+ years,MORTGAGE,180000.0,Not Verified,Dec-2018,Fully Paid,n,credit_card,Credit card refinancing,117xx,NY,11.47,0,Jul-1995,0,39.0,NaN,10,0,29711,66.8%,19,w,0.00,0.00,31804.529849,31804.53,27975.00,3829.53,0.0,0.00,0.0000,Jan-2020,20288.02,NaN,Mar-2020,0,NaN,1,Individual,NaN,...,8,13,7,10,0.0,0,0,0,78.9,60.0,0,0,286525,45387,29500,25025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,Conductor
1,10000,10000,10000.0,60 months,12.98%,227.43,B,B5,Printer,9 years,RENT,60000.0,Not Verified,Dec-2018,Current,n,credit_card,Credit card refinancing,660xx,KS,14.90,0,May-2007,0,NaN,112.0,7,1,10677,54.2%,12,w,7352.95,7352.95,4487.310000,4487.31,2647.05,1840.26,0.0,0.00,0.0000,Aug-2020,227.43,Sep-2020,Aug-2020,0,NaN,1,Individual,NaN,...,5,6,5,7,0.0,0,0,1,100.0,33.3,1,0,36200,27595,13000,16500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,Printer
2,28000,28000,28000.0,60 months,13.56%,645.15,C,C1,Project Manager,10+ years,MORTGAGE,128500.0,Source Verified,Dec-2018,Current,n,credit_card,Credit card refinancing,760xx,TX,27.35,1,Jul-1987,0,24.0,NaN,16,0,55206,75%,31,w,20667.91,20667.91,12871.360000,12871.36,7332.09,5539.27,0.0,0.00,0.0000,Aug-2020,645.15,Sep-2020,Aug-2020,0,71.0,1,Individual,NaN,...,11,17,11,15,NaN,0,0,1,94.0,100.0,0,0,542027,128345,55000,60331,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,Project Manager
3,16000,16000,16000.0,60 months,13.56%,368.66,C,C1,LMSW,10+ years,RENT,46000.0,Not Verified,Dec-2018,Current,n,credit_card,Credit card refinancing,787xx,TX,11.09,1,Oct-1997,1,11.0,NaN,9,0,18946,32.1%,21,w,11503.84,11503.84,7570.880000,7570.88,4496.16,3074.72,0.0,0.00,0.0000,Aug-2020,368.66,Sep-2020,Aug-2020,0,NaN,1,Individual,NaN,...,7,11,3,9,0.0,0,0,1,94.7,20.0,0,0,138125,128218,57600,79125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,Lmsw
4,7500,7500,7500.0,36 months,10.72%,244.55,B,B2,Program Manager,2 years,RENT,84000.0,Not Verified,Dec-2018,Charged Off,n,debt_consolidation,Debt consolidation,600xx,IL,3.86,0,M

In [ ]:
# list comprehensions can combine function calls and for loops over lists
# into one succinct and fairly readable single line of code.



['Owner', 'Supervisor', 'Project Manager', 'Teacher', 'Unknown']

In [ ]:
# We have a function that works as expected. Lets apply it to our column.
# This time we'll overwrite the original column


,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,...,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,emp_title_cleaned
0,27975,27975,27975.0,36 months,14.47%,962.52,C,C2,Conductor,10+ years,MORTGAGE,180000.0,Not Verified,Dec-2018,Fully Paid,n,credit_card,Credit card refinancing,117xx,NY,11.47,0,Jul-1995,0,39.0,NaN,10,0,29711,66.8%,19,w,0.00,0.00,31804.529849,31804.53,27975.00,3829.53,0.0,0.00,0.0000,Jan-2020,20288.02,NaN,Mar-2020,0,NaN,1,Individual,NaN,...,8,13,7,10,0.0,0,0,0,78.9,60.0,0,0,286525,45387,29500,25025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,Conductor
1,10000,10000,10000.0,60 months,12.98%,227.43,B,B5,Printer,9 years,RENT,60000.0,Not Verified,Dec-2018,Current,n,credit_card,Credit card refinancing,660xx,KS,14.90,0,May-2007,0,NaN,112.0,7,1,10677,54.2%,12,w,7352.95,7352.95,4487.310000,4487.31,2647.05,1840.26,0.0,0.00,0.0000,Aug-2020,227.43,Sep-2020,Aug-2020,0,NaN,1,Individual,NaN,...,5,6,5,7,0.0,0,0,1,100.0,33.3,1,0,36200,27595,13000,16500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,Printer
2,28000,28000,28000.0,60 months,13.56%,645.15,C,C1,Project Manager,10+ years,MORTGAGE,128500.0,Source Verified,Dec-2018,Current,n,credit_card,Credit card refinancing,760xx,TX,27.35,1,Jul-1987,0,24.0,NaN,16,0,55206,75%,31,w,20667.91,20667.91,12871.360000,12871.36,7332.09,5539.27,0.0,0.00,0.0000,Aug-2020,645.15,Sep-2020,Aug-2020,0,71.0,1,Individual,NaN,...,11,17,11,15,NaN,0,0,1,94.0,100.0,0,0,542027,128345,55000,60331,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,Project Manager
3,16000,16000,16000.0,60 months,13.56%,368.66,C,C1,Lmsw,10+ years,RENT,46000.0,Not Verified,Dec-2018,Current,n,credit_card,Credit card refinancing,787xx,TX,11.09,1,Oct-1997,1,11.0,NaN,9,0,18946,32.1%,21,w,11503.84,11503.84,7570.880000,7570.88,4496.16,3074.72,0.0,0.00,0.0000,Aug-2020,368.66,Sep-2020,Aug-2020,0,NaN,1,Individual,NaN,...,7,11,3,9,0.0,0,0,1,94.7,20.0,0,0,138125,128218,57600,79125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,Lmsw
4,7500,7500,7500.0,36 months,10.72%,244.55,B,B2,Program Manager,2 years,RENT,84000.0,Not Verified,Dec-2018,Charged Off,n,debt_consolidation,Debt consolidation,600xx,IL,3.86,0,M

We can use the same code as we did earlier to see how much progress was made.


In [ ]:
# Look at the top 20 employment titles

df['emp_title_cleaned'].value_counts(dropna=False).sort_values(ascending=False).head(20)

Unknown               20947
Teacher                2557
Manager                2395
Registered Nurse       1418
Driver                 1258
Supervisor             1160
Truck Driver            920
Rn                      834
Office Manager          805
Sales                   803
General Manager         791
Project Manager         720
Owner                   625
Director                523
Operations Manager      518
Sales Manager           500
Police Officer          440
Nurse                   425
Technician              420
Engineer                412
Name: emp_title_cleaned, dtype: int64

In [ ]:
# How many different unique employment titles are there currently?



34902

In [ ]:
# How often is the employment_title null (NaN)?



0

## Challenge

Using the .apply() function isn't always about creating new columns on a dataframe, we can use it to clean up or modify existing columns as well. 

# [Objective](#dates-and-times) Work with Dates and Times with Pandas

## Overview

Pandas has its own datatype datatype that makes it extremely convenient to convert strings that are in standard date formates to datetime objects and then use those datetime objects to either create new features on a dataframe or work with the dataset in a timeseries fashion. 

This section will demonstrate how to take a column of date strings, convert it to a datetime object and then use the datetime formatting `.dt` to access specific parts of the date (year, month, day) to generate useful columns on a dataframe.

## Follow Along

### Work with Dates 

pandas documentation
- [to_datetime](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.to_datetime.html)
- [Time/Date Components](https://pandas.pydata.org/pandas-docs/stable/timeseries.html#time-date-components) "You can access these properties via the `.dt` accessor"

Many of the most useful date columns in this dataset have the suffix `_d` to indicate that they correspond to dates.

We'll use a list comprehension to print them out

In [ ]:
[col for col in df if col.endswith('_d')]

['issue_d', 'last_pymnt_d', 'next_pymnt_d', 'last_credit_pull_d']

Lets look at the string format of the `issue_d` column

0    Dec-2018
1    Dec-2018
2    Dec-2018
3    Dec-2018
4    Dec-2018
Name: issue_d, dtype: object

Because this string format %m-%y is a common datetime format, we can just let Pandas detect this format and translate it to the appropriate datetime object.

'Dec-2018'

Now we can see that the `issue_d` column has been changed to hold `datetime` objects.

Lets look at one of the cells specifically to see what a datetime object looks like:

0        2018-12-01
1        2018-12-01
2        2018-12-01
3        2018-12-01
4        2018-12-01
            ...    
128407   2018-10-01
128408   2018-10-01
128409   2018-10-01
128410   2018-10-01
128411   2018-10-01
Name: issue_d, Length: 128412, dtype: datetime64[ns]

You can see how the month and year have been indicated by the strings that were contained in the column previously, and that the rest of the values have been inferred.

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,...,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,emp_title_cleaned
0,27975,27975,27975.0,36 months,14.47%,962.52,C,C2,Conductor,10+ years,MORTGAGE,180000.0,Not Verified,2018-12-01,Fully Paid,n,credit_card,Credit card refinancing,117xx,NY,11.47,0,Jul-1995,0,39.0,NaN,10,0,29711,66.8%,19,w,0.00,0.00,31804.529849,31804.53,27975.00,3829.53,0.0,0.00,0.0000,Jan-2020,20288.02,NaN,Mar-2020,0,NaN,1,Individual,NaN,...,8,13,7,10,0.0,0,0,0,78.9,60.0,0,0,286525,45387,29500,25025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,Conductor
1,10000,10000,10000.0,60 months,12.98%,227.43,B,B5,Printer,9 years,RENT,60000.0,Not Verified,2018-12-01,Current,n,credit_card,Credit card refinancing,660xx,KS,14.90,0,May-2007,0,NaN,112.0,7,1,10677,54.2%,12,w,7352.95,7352.95,4487.310000,4487.31,2647.05,1840.26,0.0,0.00,0.0000,Aug-2020,227.43,Sep-2020,Aug-2020,0,NaN,1,Individual,NaN,...,5,6,5,7,0.0,0,0,1,100.0,33.3,1,0,36200,27595,13000,16500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,Printer
2,28000,28000,28000.0,60 months,13.56%,645.15,C,C1,Project Manager,10+ years,MORTGAGE,128500.0,Source Verified,2018-12-01,Current,n,credit_card,Credit card refinancing,760xx,TX,27.35,1,Jul-1987,0,24.0,NaN,16,0,55206,75%,31,w,20667.91,20667.91,12871.360000,12871.36,7332.09,5539.27,0.0,0.00,0.0000,Aug-2020,645.15,Sep-2020,Aug-2020,0,71.0,1,Individual,NaN,...,11,17,11,15,NaN,0,0,1,94.0,100.0,0,0,542027,128345,55000,60331,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,Project Manager
3,16000,16000,16000.0,60 months,13.56%,368.66,C,C1,Lmsw,10+ years,RENT,46000.0,Not Verified,2018-12-01,Current,n,credit_card,Credit card refinancing,787xx,TX,11.09,1,Oct-1997,1,11.0,NaN,9,0,18946,32.1%,21,w,11503.84,11503.84,7570.880000,7570.88,4496.16,3074.72,0.0,0.00,0.0000,Aug-2020,368.66,Sep-2020,Aug-2020,0,NaN,1,Individual,NaN,...,7,11,3,9,0.0,0,0,1,94.7,20.0,0,0,138125,128218,57600,79125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,Lmsw
4,7500,7500,7500.0,36 months,10.72%,244.55,B,B2,Program Manager,2 years,RENT,84000.0,Not Verified,2018-12-01,Charged Off,n,debt_consolidation,Debt consolidation,600xx,I

We can use the `.dt` accessor to now grab specific parts of the datetime object. Lets grab just the year from the all of the cells in the `issue_d` column

0         2018
1         2018
2         2018
3         2018
4         2018
          ... 
128407    2018
128408    2018
128409    2018
128410    2018
128411    2018
Name: issue_d, Length: 128412, dtype: int64

Now the month.

0         12
1         12
2         12
3         12
4         12
          ..
128407    10
128408    10
128409    10
128410    10
128411    10
Name: issue_d, Length: 128412, dtype: int64

It's just that easy! Now, instead of printing them out, lets add these year and month values as new columns on our dataframe. Again, you'll have to scroll all the way over to the right in the table to see the new columns.

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,...,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,emp_title_cleaned,issue_year,issue_month
0,27975,27975,27975.0,36 months,14.47%,962.52,C,C2,Conductor,10+ years,MORTGAGE,180000.0,Not Verified,2018-12-01,Fully Paid,n,credit_card,Credit card refinancing,117xx,NY,11.47,0,Jul-1995,0,39.0,NaN,10,0,29711,66.8%,19,w,0.00,0.00,31804.529849,31804.53,27975.00,3829.53,0.0,0.00,0.0000,Jan-2020,20288.02,NaN,Mar-2020,0,NaN,1,Individual,NaN,...,7,10,0.0,0,0,0,78.9,60.0,0,0,286525,45387,29500,25025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,Conductor,2018,12
1,10000,10000,10000.0,60 months,12.98%,227.43,B,B5,Printer,9 years,RENT,60000.0,Not Verified,2018-12-01,Current,n,credit_card,Credit card refinancing,660xx,KS,14.90,0,May-2007,0,NaN,112.0,7,1,10677,54.2%,12,w,7352.95,7352.95,4487.310000,4487.31,2647.05,1840.26,0.0,0.00,0.0000,Aug-2020,227.43,Sep-2020,Aug-2020,0,NaN,1,Individual,NaN,...,5,7,0.0,0,0,1,100.0,33.3,1,0,36200,27595,13000,16500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,Printer,2018,12
2,28000,28000,28000.0,60 months,13.56%,645.15,C,C1,Project Manager,10+ years,MORTGAGE,128500.0,Source Verified,2018-12-01,Current,n,credit_card,Credit card refinancing,760xx,TX,27.35,1,Jul-1987,0,24.0,NaN,16,0,55206,75%,31,w,20667.91,20667.91,12871.360000,12871.36,7332.09,5539.27,0.0,0.00,0.0000,Aug-2020,645.15,Sep-2020,Aug-2020,0,71.0,1,Individual,NaN,...,11,15,NaN,0,0,1,94.0,100.0,0,0,542027,128345,55000,60331,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,Project Manager,2018,12
3,16000,16000,16000.0,60 months,13.56%,368.66,C,C1,Lmsw,10+ years,RENT,46000.0,Not Verified,2018-12-01,Current,n,credit_card,Credit card refinancing,787xx,TX,11.09,1,Oct-1997,1,11.0,NaN,9,0,18946,32.1%,21,w,11503.84,11503.84,7570.880000,7570.88,4496.16,3074.72,0.0,0.00,0.0000,Aug-2020,368.66,Sep-2020,Aug-2020,0,NaN,1,Individual,NaN,...,3,9,0.0,0,0,1,94.7,20.0,0,0,138125,128218,57600,79125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,Lmsw,2018,12
4,7500,7500,7500.0,36 months,10.72%,244.55,B,B2,Program Manager,2 years,RENT,84000.0,Not Verified,2018-12-01,Charged Off,n,debt_consolidation,Debt consolidation,

Because all of these dates come from Q4 of 2018, the `issue_d` column isn't all that interesting. Lets look at the `earliest_cr_line` column, which is also a string, but that could be converted to datetime format.

We're going to create a new column called `days_from_earliest_credit_to_issue`

It's a long column header, but think about how valuable this piece of information could be. This number will essentially indicate the length of a person's credit history and if that is correlated with repayment or other factors could be a valuable predictor!

0         8554 days
1         4232 days
2        11476 days
3         7731 days
4         4962 days
            ...    
128407   12114 days
128408   13241 days
128409    4505 days
128410    3652 days
128411    4413 days
Length: 128412, dtype: timedelta64[ns]

In [ ]:


#Don't forget leap year = 365.25 each year.



,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,...,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,emp_title_cleaned,issue_year,issue_month,credit_length_days,credit_length_years
0,27975,27975,27975.0,36 months,14.47%,962.52,C,C2,Conductor,10+ years,MORTGAGE,180000.0,Not Verified,2018-12-01,Fully Paid,n,credit_card,Credit card refinancing,117xx,NY,11.47,0,1995-07-01,0,39.0,NaN,10,0,29711,66.8%,19,w,0.00,0.00,31804.529849,31804.53,27975.00,3829.53,0.0,0.00,0.0000,Jan-2020,20288.02,NaN,Mar-2020,0,NaN,1,Individual,NaN,...,0.0,0,0,0,78.9,60.0,0,0,286525,45387,29500,25025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,Conductor,2018,12,8554,23.419576
1,10000,10000,10000.0,60 months,12.98%,227.43,B,B5,Printer,9 years,RENT,60000.0,Not Verified,2018-12-01,Current,n,credit_card,Credit card refinancing,660xx,KS,14.90,0,2007-05-01,0,NaN,112.0,7,1,10677,54.2%,12,w,7352.95,7352.95,4487.310000,4487.31,2647.05,1840.26,0.0,0.00,0.0000,Aug-2020,227.43,Sep-2020,Aug-2020,0,NaN,1,Individual,NaN,...,0.0,0,0,1,100.0,33.3,1,0,36200,27595,13000,16500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,Printer,2018,12,4232,11.586585
2,28000,28000,28000.0,60 months,13.56%,645.15,C,C1,Project Manager,10+ years,MORTGAGE,128500.0,Source Verified,2018-12-01,Current,n,credit_card,Credit card refinancing,760xx,TX,27.35,1,1987-07-01,0,24.0,NaN,16,0,55206,75%,31,w,20667.91,20667.91,12871.360000,12871.36,7332.09,5539.27,0.0,0.00,0.0000,Aug-2020,645.15,Sep-2020,Aug-2020,0,71.0,1,Individual,NaN,...,NaN,0,0,1,94.0,100.0,0,0,542027,128345,55000,60331,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,Project Manager,2018,12,11476,31.419576
3,16000,16000,16000.0,60 months,13.56%,368.66,C,C1,Lmsw,10+ years,RENT,46000.0,Not Verified,2018-12-01,Current,n,credit_card,Credit card refinancing,787xx,TX,11.09,1,1997-10-01,1,11.0,NaN,9,0,18946,32.1%,21,w,11503.84,11503.84,7570.880000,7570.88,4496.16,3074.72,0.0,0.00,0.0000,Aug-2020,368.66,Sep-2020,Aug-2020,0,NaN,1,Individual,NaN,...,0.0,0,0,1,94.7,20.0,0,0,138125,128218,57600,79125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,Lmsw,2018,12,7731,21.166324
4,7500,7500,7500.0,36 months,10.72%,244.55,B,B2,Program Manager,2 years,RENT,84000.0,Not Verified,201

What we're about to do is so cool! Pandas' datetime format is so smart that we can simply use the subtraction operator `-` in order to calculate the amount of time between two dates. 

Think about everything that's going on under the hood in order to give us such straightforward syntax! Handling months of different lengths, leap years, etc. Pandas datetime objects are seriously powerful!

68.91444216290212

What's oldest credit history that was involved in Q4 2018? 

68.91444216290212

25,171 days is ~ 68.96 years of credit history!

## Challenge

Pandas' datetime format is so easy to work with that there's really no excuse for not using dates to make features on a dataframe! Get ready to practice more of this on your assignment.